In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import lightgbm as lgb
from matplotlib import pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

from ctgan import CTGAN

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

import mlflow
import mlflow.sklearn
import mlflow.xgboost

mlflow.set_tracking_uri("http://localhost:5000")

# Explorative Data Analysis

In [ ]:
# df = pd.read_excel(r'data\flats_to_rent_wue_preprocessed_combined.xlsx')
# profile = ProfileReport(df, title="Flats -  Würzburg - Rent - Overview", explorative=True)
# profile.to_file("eda-wue-rent-all.html")

In [ ]:
# df = pd.read_excel(r'data\houses_to_buy_wue_preprocessed_1207.xlsx')
# profile = ProfileReport(df, title="Houses -  Würzburg - Buy - Overview", explorative=True)
# profile.to_file("eda-wue-houses.html")

# Preprocessing & Feature Engineering


In [ ]:
def determineHighCorrCols(df):
    df.columns = [re.sub(r"\\u([0-9a-fA-F]{4})", lambda m: chr(int(m.group(1), 16)), col) for col in df.columns]
    df.columns = [col.replace("ö", "oe").replace("ä", "ae").replace("ü", "ue").replace("ß", "ss") for col in df.columns]
    important_num_cols = list(df.corr()["Object_price"][(df.corr()["Object_price"]>0.20) | (df.corr()["Object_price"]<-0.20)].index)
    cat_cols = [col for col in df.columns if df[col].dtype == 'object']
    important_cols = important_num_cols + cat_cols +   ["ConstructionYear"] + ["ZipCode"]
    #   df = df[important_cols]
    print(important_cols)
    return important_cols

In [ ]:
def preprocess_data(df, feature_set):
    print(f"Used feature set for preprocessing:{feature_set}")
    df.columns = [
        re.sub(r"\\u([0-9a-fA-F]{4})", lambda m: chr(int(m.group(1), 16)), col)
        for col in df.columns
    ]
    df.columns = [
        col.replace("ö", "oe").replace("ä", "ae").replace("ü", "ue").replace("ß", "ss")
        for col in df.columns
    ]

    df = df.replace('""', np.nan)
    df = df.dropna()
    df["LivingSpace"] = df["LivingSpace"].astype(float)
    df["Rooms"] = df["Rooms"].astype(float)
    df = df.dropna(subset=["ConstructionYear"])
    df = df.dropna(subset=["Object_price"])
    df = df.dropna(subset=["Rooms"])
    df = df.dropna(subset=["LivingSpace"])
    df["ZipCode"] = df["ZipCode"].astype(str)
    df["LivingSpace"] = df["LivingSpace"].astype(float)
    df = df[feature_set]
    df = df.reindex()
    df = df.reset_index(drop=True)

    # one-hot encoding
    cat_cols = [col for col in df.columns if df[col].dtype == "object"]
    y = df["Object_price"]
    X = df.drop("Object_price", axis=1)
    X = pd.get_dummies(X, columns=cat_cols)

    return X, y

# Data Split


In [ ]:
def data_split(X, y, train_size=0.8, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_size, random_state=random_state
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, train_size=train_size, random_state=random_state
    )

    # X_train.to_excel("data/X_train.xlsx")
    # X_val.to_excel("data/X_val.xlsx")
    # X_test.to_excel("data/X_test.xlsx")
    # y_train.to_excel("data/y_train.xlsx")
    # y_val.to_excel("data/y_val.xlsx")
    # y_test.to_excel("data/y_test.xlsx")

    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
df = pd.read_excel(r"data\flats_to_rent_wue_preprocessed_combined.xlsx")

X = df.drop("Object_price", axis=1)
y = df["Object_price"]

data_split(X, y)

# Training, Evaluation and Logging Of Models

## Baseline 

In [ ]:
def scrape_avg_rental_prices():
    url = "https://www.wohnungsboerse.net/mietspiegel-Wuerzburg/2772"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    script_tag = soup.find("script", string=lambda text: "pdfData" in text)
    rental_price = 0
    if script_tag:
        script_content = script_tag.string
        start_index = script_content.find("avg_rent_price: ") + len("avg_rent_price: '")
        end_index = script_content.find("',", start_index)
        rental_price = script_content[start_index:end_index]
        rental_price = (
            rental_price.replace("€/m2", "").replace(".", "").replace(",", ".")
        )
        rental_price = rental_price.strip()
        rental_price = float(rental_price)
        print(f"Extrcated rental price per square meter via scraper: {rental_price}")
    else:
        print("The script tag containing the rental price was not found.")
    return rental_price

In [ ]:
def scrape_avg_buy_prices():
    url = "https://www.wohnungsboerse.net/immobilienpreise-Wuerzburg/2772"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    p_element = soup.find("p", class_="mb-8")
    buy_price = 0
    if p_element:
        pattern = r"\d{1,3}(?:\.\d{3})*(?:,\d{2})?€/m²"
        match = re.search(pattern, p_element.text)
        if match:
            buy_price = match.group()
            buy_price = buy_price.replace("€/m²", "").replace(".", "").replace(",", ".")
            print(f"Extrcated buy price per square meter via scraper: {buy_price}")
        else:
            print("Price not found")
    else:
        print("The element ontaining the buy price was not found.")
    return buy_price

In [ ]:
def baseline_rent(val_X, val_y, runname="baseline_rent"):
    avg_price_per_sqm_rent = scrape_avg_rental_prices()
    print(f"Average rental price per sqm: {avg_price_per_sqm_rent}")
    baseline_preds = val_X["LivingSpace"] * avg_price_per_sqm_rent
    baseline_mae = mean_absolute_error(val_y, baseline_preds)
    baseline_r2 = r2_score(val_y, baseline_preds)
    baseline_mse = mean_squared_error(val_y, baseline_preds)

    with mlflow.start_run(run_name=runname):
        mlflow.log_metric("mse", baseline_mse)
        mlflow.log_metric("mae", baseline_mae)
        mlflow.log_metric("r2", baseline_r2)

    print(f"Baseline Mae: {baseline_mae}")
    print(f"Baseline MSE: {baseline_mse}")
    print(f"Baseline R2 Score: {baseline_r2}")
    return avg_price_per_sqm_rent, baseline_mae, baseline_mse, baseline_r2

In [ ]:
def baseline_buy(X_val, y_val, runname="baseline_buy"):
    avg_price_per_sqm_buy = scrape_avg_buy_prices()
    print(f"Average rental price per sqm: {avg_price_per_sqm_buy}")

    baseline_preds = X_val["LivingSpace"] * avg_price_per_sqm_buy
    baseline_mae = mean_absolute_error(y_val, baseline_preds)
    baseline_r2 = r2_score(y_val, baseline_preds)
    baseline_mse = mean_squared_error(y_val, baseline_preds)

    with mlflow.start_run(run_name=runname):
        mlflow.log_metric("mse", baseline_mse)
        mlflow.log_metric("mae", baseline_mae)
        mlflow.log_metric("r2", baseline_r2)

    print(f"Baseline Mae: {baseline_mae}")
    print(f"Baseline MSE: {baseline_mse}")
    print(f"Baseline R2 Score: {baseline_r2}")

    return avg_price_per_sqm_buy, baseline_mae, baseline_mse, baseline_r2

## Linear Regression

In [ ]:
def train_and_eval_linear(X_train, y_train, X_val, y_val, runname="linear-regression"):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name=runname):
        model = LinearRegression()
        model.fit(X_train, y_train)

        pred_train = model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred_train)
        mse_train = mean_squared_error(y_train, pred_train)
        r2_train = r2_score(y_train, pred_train)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)

        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mlflow.log_metric("mae", mae_train)
        mlflow.log_metric("mse", mse_train)
        mlflow.log_metric("r2", r2_train)
    return model, mae, mse, r2, mae_train, mse_train, r2_train

## Lasso Regression

In [ ]:
def train_and_eval_lasso(X_train, y_train, X_val, y_val, runname="lasso-regression"):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name=runname):
        model = Lasso()
        model.fit(X_train, y_train)

        pred_train = model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred_train)
        mse_train = mean_squared_error(y_train, pred_train)
        r2_train = r2_score(y_train, pred_train)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)

        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mlflow.log_metric("mae", mae_train)
        mlflow.log_metric("mse", mse_train)
        mlflow.log_metric("r2", r2_train)
    return model, mae, mse, r2, mae_train, mse_train, r2_train

## Ridge Regression

In [ ]:
def train_and_eval_ridge(X_train, y_train, X_val, y_val, runname="ridge-regression"):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name=runname):
        model = Ridge()
        model.fit(X_train, y_train)

        pred_train = model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred_train)
        mse_train = mean_squared_error(y_train, pred_train)
        r2_train = r2_score(y_train, pred_train)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)

        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mlflow.log_metric("mae", mae_train)
        mlflow.log_metric("mse", mse_train)
        mlflow.log_metric("r2", r2_train)
    return model, mae, mse, r2, mae_train, mse_train, r2_train

## Random Forest

In [ ]:
def train_and_eval_rf(
    X_train,
    y_train,
    X_val,
    y_val,
    n_estimators=50,
    random_state=0,
    run_name="random-forest",
):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name=run_name):
        model = RandomForestRegressor(
            n_estimators=n_estimators, random_state=random_state
        )
        model.fit(X_train, y_train)
        pred_train = model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred_train)
        mse_train = mean_squared_error(y_train, pred_train)
        r2_train = r2_score(y_train, pred_train)
        preds = model.predict(X_val)
        mae = mean_absolute_error(y_val, preds)
        mse = mean_squared_error(y_val, preds)
        r2 = r2_score(y_val, preds)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)
        return model, mae, mse, r2, mae_train, mse_train, r2_train

## XGBRegressor


In [ ]:
def train_and_eval_xgb(
    X_train,
    y_train,
    X_val,
    y_val,
    run_name="xgb",
    early_stopping_rounds=30,
    max_depth=6,
    n_estimators=1000,
):
    mlflow.xgboost.autolog()
    with mlflow.start_run(run_name=run_name):
        model = xgb.XGBRegressor(
            eval_metric=["rmse", "mae"],
            early_stopping_rounds=early_stopping_rounds,
            random_state=42,
            max_depth=max_depth,
            n_estimators=n_estimators,
        )
        mlflow.log_param("early_stopping_rounds", early_stopping_rounds)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("n_estimators", n_estimators)
        model.fit(X=X_train, y=y_train, eval_set=[(X_val, y_val)], verbose=True)
        pred_train = model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred_train)
        mse_train = mean_squared_error(y_train, pred_train)
        r2_train = r2_score(y_train, pred_train)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)
        preds = model.predict(X_val)
        mae = mean_absolute_error(y_val, preds)
        mse = mean_squared_error(y_val, preds)
        r2 = r2_score(y_val, preds)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        return model, mae, mse, r2, mae_train, mse_train, r2_train

## ElasticNet

In [ ]:
def train_and_eval_elasticnet(X_train, y_train, X_val, y_val, runname="elasticNet"):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name=runname):
        model = ElasticNet()
        model.fit(X_train, y_train)
        pred_train = model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred_train)
        mse_train = mean_squared_error(y_train, pred_train)
        r2_train = r2_score(y_train, pred_train)
        preds = model.predict(X_val)
        mae = mean_absolute_error(y_val, preds)
        mse = mean_squared_error(y_val, preds)
        r2 = r2_score(y_val, preds)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)
        return model, mae, mse, r2, mae_train, mse_train, r2_train

# Complete Pipeline

In [ ]:
def pipeline_from_extracted(df, feature_set, model_name="lasso"):
    X, y = preprocess_data(df, feature_set)
    print("Done with preprocessing.")
    X_train, y_train, X_val, y_val, test_X, test_y = data_split(X, y)
    print("Done with data split")
    model, mae, mse, r2, mae_train, mse_train, r2_train = (
        None,
        None,
        None,
        None,
        None,
        None,
        None,
    )

    if model_name == "lasso":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_lasso(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "ridge":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_ridge(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "rf":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_rf(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "xgb":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_xgb(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "elasticnet":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_elasticnet(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "linear":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_linear(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "baseline-rent":
        avg_price, mae, mse, r2 = baseline_rent(X_val, y_val)
    else:
        print(
            "Model not found. Model_name must be 'lasso', 'ridge', 'rf', 'xgb', 'elasticnet', 'linear', 'baseline_buy' or 'baseline_rent'"
        )
    print("Done with train")
    return model, mae, mse, r2, mae_train, mse_train, r2_train

# Complete Pipeline

### Regular Pipeline

In [ ]:
def pipeline_from_extracted(df, feature_set, model_name="lasso"):
    X, y = preprocess_data(df, feature_set)
    print("Done with preprocessing")
    X_train, y_train, X_val, y_val, test_X, test_y = data_split(X,y)
    print("Done with data split")

    model, mae, mse, r2, mae_train, mse_train, r2_train = None, None, None, None, None, None, None

    if model_name == "lasso":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_lasso(X_train, y_train, X_val, y_val)
    elif model_name == "ridge":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_ridge(X_train, y_train, X_val, y_val)
    elif model_name == "rf":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_rf(X_train, y_train, X_val, y_val)
    elif model_name == "xgb":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_xgb(X_train, y_train, X_val, y_val)
    elif model_name == "elasticnet":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_elasticnet(X_train, y_train, X_val, y_val)
    elif model_name == "linear":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_linear(X_train, y_train, X_val, y_val)
    elif model_name =="baseline-rent":
        avg_price, mae, mse, r2 = baseline_rent(X_val, y_val)
    else:
        print("Model not found. Model_name must be 'lasso', 'ridge', 'rf', 'xgb', 'elasticnet', 'linear', 'baseline_buy' or 'baseline_rent'")
    print("Done with train")
    return model, mae, mse, r2, mae_train, mse_train, r2_train

### Augmented Data Approach With CTGan

In [35]:
def complete_from_extracted_with_augemented_training_data(
    df,
    feature_set,
    model_name="lasso",
    augmentation_epochs=30,
    n_added_fake_train_data=1000,
):
    df = pd.read_excel(r"data\flats_to_rent_wue_preprocessed_combined.xlsx")
    X = df.drop(columns=["Object_price"])
    y = df["Object_price"]
    X_train, y_train, X_val, y_val, X_test, y_test = data_split(X, y)
    train_extended = pd.concat(
        [X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1
    )
    val = pd.concat(
        [X_val.reset_index(drop=True), y_val.reset_index(drop=True)], axis=1
    )

    df_cttrain = X_train.copy()
    df_cttrain["Object_price"] = y_train
    df_cttrain = df_cttrain[feature_set]
    df_cttrain = df_cttrain.dropna()

    #numerical_features = ["ConstructionYear", "Rooms", "LivingSpace", "Object_price"]
    numerical_features = ["Rooms", "LivingSpace", "Object_price"]

    df_cttrain[numerical_features] = df_cttrain[numerical_features].astype("int64")
    categorical_features = [
        col for col in df_cttrain.columns if col not in numerical_features
    ]
    ctgan = CTGAN(verbose=True)
    ctgan.fit(df_cttrain, categorical_features, epochs=augmentation_epochs)

    samples = ctgan.sample(n_added_fake_train_data * 3)
    samples["Object_price"] = samples["Object_price"].clip(lower=0)
    #samples = samples[samples["ConstructionYear"] < 2023]
    samples = samples[samples["LivingSpace"] > 20]
    samples = samples[samples["Object_price"] > 120]
    samples = samples.head(n_added_fake_train_data)

    X_sample_train = samples.drop(columns=["Object_price"])
    y_sample_train = samples["Object_price"]
    X_train = pd.concat([X_train, X_sample_train])
    y_train = pd.concat([y_train, y_sample_train])

    train_extended = pd.concat(
        [X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1
    )
    X_train, y_train = preprocess_data(train_extended, feature_set_selected)
    X_val, y_val = preprocess_data(val, feature_set_selected)

    X_train, X_val = X_train.align(X_val, join="outer", axis=1, fill_value=0)

    model, mae, mse, r2, mae_train, mse_train, r2_train = (
        None,
        None,
        None,
        None,
        None,
        None,
        None,
    )

    if model_name == "lasso":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_lasso(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "ridge":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_ridge(
            X_train,
            y_train,
            X_val,
            y_val,
        )
    elif model_name == "rf":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_rf(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "xgb":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_xgb(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "elasticnet":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_elasticnet(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "linear":
        model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_linear(
            X_train, y_train, X_val, y_val
        )
    elif model_name == "baseline-rent":
        avg_price, mae, mse, r2 = baseline_rent(X_val, y_val)
    else:
        print(
            "Model not found. Model_name must be 'lasso', 'ridge', 'rf', 'xgb', 'elasticnet', 'linear', 'baseline_buy' or 'baseline_rent'"
        )
    print("Done with train")
    return model, mae, mse, r2, mae_train, mse_train, r2_train

# Execution of complete pipeline

In [ ]:
df = pd.read_excel(r'data\flats_to_rent_wue_preprocessed_combined.xlsx')
feature_set_high_corr = determineHighCorrCols(df) + ["ZipCode"]

In [36]:
mlflow.set_experiment("wue-rent-selected-features-aug")
model_names = ['lasso', 'ridge', 'rf', 'xgb', 'elasticnet', 'linear', 'baseline-rent']

feature_set_selected = [
    "Object_price",
    "LivingSpace",
    "ZipCode", 
    "Rooms",
    "altbau_(bis_1945)",
    "balkon",
    "barriefrei",
    "dachgeschoss",
    "einbaukueche",
    "neubau",
    "parkett",
    "stellplatz", 
    "bad/wc_getrennt",
    "personenaufzug",
    "garten",
    "garage",
    "renoviert", 
    "terrasse", 
    "wanne", 
    "zentralheizung",
    "abstellraum",
    "ferne",
    "fussbodenheizung",
    "gartennutzung",
    "kelleranteil"
]

df = pd.read_excel(r'data\flats_to_rent_wue_preprocessed_combined.xlsx')
    
# for model_name in model_names:
#     df = pd.read_excel(r'data\flats_to_rent_wue_preprocessed_combined.xlsx')
#     print("Starting with model: ", model_name)
#     pipeline_from_extracted(df, model_name=model_name, feature_set=feature_set_selected)
#     print("Done with model: ", model_name)
#     print("--------------------------------------------------------")

for model_name in model_names:
    df = pd.read_excel(r'data\flats_to_rent_wue_preprocessed_combined.xlsx')
    print("Starting with model: ", model_name)
    complete_from_extracted_with_augemented_training_data(df, model_name=model_name, feature_set=feature_set_selected, n_added_fake_train_data=1000)
    print("Done with model: ", model_name)
    print("--------------------------------------------------------")

Starting with model:  lasso


c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
2023/07/13 16:09:09 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b487fad9c031440e85615b1d7916591f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/07/13 16:09:14 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'm

Epoch 1, Loss G:  0.9356,Loss D:  0.0132
Epoch 2, Loss G:  0.8102,Loss D: -0.0319
Epoch 3, Loss G:  0.8747,Loss D: -0.0437
Epoch 4, Loss G:  0.8638,Loss D: -0.0921
Epoch 5, Loss G:  0.8298,Loss D: -0.1069
Epoch 6, Loss G:  0.8150,Loss D: -0.1364
Epoch 7, Loss G:  0.8580,Loss D: -0.2187
Epoch 8, Loss G:  0.8764,Loss D: -0.1881
Epoch 9, Loss G:  0.8820,Loss D: -0.2984
Epoch 10, Loss G:  0.7535,Loss D: -0.2945
Epoch 11, Loss G:  0.6756,Loss D: -0.3318
Epoch 12, Loss G:  0.6349,Loss D: -0.4214
Epoch 13, Loss G:  0.5858,Loss D: -0.3835
Epoch 14, Loss G:  0.5471,Loss D: -0.4964
Epoch 15, Loss G:  0.4915,Loss D: -0.4295
Epoch 16, Loss G:  0.3013,Loss D: -0.3521
Epoch 17, Loss G:  0.2984,Loss D: -0.3796
Epoch 18, Loss G:  0.2261,Loss D: -0.3635
Epoch 19, Loss G:  0.2516,Loss D: -0.3230
Epoch 20, Loss G:  0.1268,Loss D: -0.3015
Epoch 21, Loss G:  0.0283,Loss D: -0.1586
Epoch 22, Loss G: -0.1515,Loss D: -0.1327
Epoch 23, Loss G: -0.0641,Loss D: -0.1824
Epoch 24, Loss G: -0.1546,Loss D: -0.1765
E

2023/07/13 16:10:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/13 16:10:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\env

Done with train
Done with model:  lasso
--------------------------------------------------------
Starting with model:  ridge


...

c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
2023/07/13 16:13:31 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '72d0ee62b6b3496497a1195d1e3f3c57', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/07/13 16:13:35 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'm

Epoch 1, Loss G:  0.7787,Loss D:  0.0006
Epoch 2, Loss G:  0.7542,Loss D: -0.0000
Epoch 3, Loss G:  0.7276,Loss D: -0.0502
Epoch 4, Loss G:  0.8083,Loss D: -0.1022
Epoch 5, Loss G:  0.7509,Loss D: -0.1024
Epoch 6, Loss G:  0.7972,Loss D: -0.1581
Epoch 7, Loss G:  0.7280,Loss D: -0.1341
Epoch 8, Loss G:  0.6097,Loss D: -0.2344
Epoch 9, Loss G:  0.7093,Loss D: -0.2888
Epoch 10, Loss G:  0.5670,Loss D: -0.2660
Epoch 11, Loss G:  0.5979,Loss D: -0.2782
Epoch 12, Loss G:  0.5079,Loss D: -0.3314
Epoch 13, Loss G:  0.4540,Loss D: -0.3686
Epoch 14, Loss G:  0.3658,Loss D: -0.3501
Epoch 15, Loss G:  0.2919,Loss D: -0.4151
Epoch 16, Loss G:  0.1176,Loss D: -0.2503
Epoch 17, Loss G:  0.0171,Loss D: -0.3003
Epoch 18, Loss G: -0.0389,Loss D: -0.1905
Epoch 19, Loss G: -0.1591,Loss D: -0.1795
Epoch 20, Loss G: -0.2679,Loss D: -0.1927
Epoch 21, Loss G: -0.2223,Loss D: -0.0701
Epoch 22, Loss G: -0.3629,Loss D: -0.1813
Epoch 23, Loss G: -0.3770,Loss D:  0.0109
Epoch 24, Loss G: -0.4688,Loss D: -0.0879
E

2023/07/13 16:14:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/13 16:15:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\env

Done with train
Done with model:  rf
--------------------------------------------------------
Starting with model:  xgb


c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
2023/07/13 16:15:42 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '91c06fd22eb3437680739a8145c143b7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/07/13 16:15:46 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\rdt\transformers\base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'm

In [ ]:
# df = pd.read_excel(r"data\flats_to_rent_wue_preprocessed_combined.xlsx")
# complete_from_extracted_with_augemented_training_data(df, feature_set_selected, model_name="lasso", augmentation_epochs=20, n_added_fake_train_data=1000)

## Get all runs and make result plots

In [ ]:
mlflow.search_runs(experiment_names=["wue-rent-high-corr-features-comparison"])
runs = mlflow.search_runs(experiment_names=["wue-rent-high-corr-features-comparison"])
mae_runs = runs[runs["metrics.mae"].notnull()]

print(mae_runs)
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(mae_runs["tags.mlflow.runName"], mae_runs["metrics.mae"], color='lightblue')
ax.set_xlabel("Model", fontsize=14)
ax.set_ylabel("MAE", fontsize=14)
ax.set_title("MAE: wue-rent-combined-high-corr-features", fontsize=16)
plt.tight_layout()
plt.show()